# Creation de la base de donnees

In [22]:
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from datetime import datetime

# Create an engine and a metadata object
engine = create_engine('sqlite:///billing_db.db')
Base = declarative_base()

# Define the Product class
class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    price = Column(Float, nullable=False)

# Define the Customer class
class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    billings = relationship('Billing', backref='customer', lazy=True, cascade="all, delete-orphan")

# Define the Billing class
class Billing(Base):
    __tablename__ = 'billings'
    id = Column(Integer, primary_key=True)
    date = Column(DateTime, nullable=False, default=datetime.utcnow)
    customer_id = Column(Integer, ForeignKey('customers.id', ondelete='CASCADE'), nullable=False)
    items = relationship('BillingItem', backref='billing', lazy=True, cascade="all, delete-orphan")

# Define the BillingItem class
class BillingItem(Base):
    __tablename__ = 'billing_items'
    id = Column(Integer, primary_key=True)
    billing_id = Column(Integer, ForeignKey('billings.id', ondelete='CASCADE'), nullable=False)
    product_id = Column(Integer, ForeignKey('products.id', ondelete='CASCADE'), nullable=False)
    quantity = Column(Integer, nullable=False)
    product = relationship('Product')

# Create the tables
Base.metadata.create_all(engine)

# Create a configured "Session" class
Session = sessionmaker(bind=engine)

# Create a Session
session = Session()

# Commit the session
session.commit()

# Close the session
session.close()

print("Database and tables created successfully.")


Database and tables created successfully.


/tmp/ipykernel_19304/3335914113.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


# Ajout de produits

In [23]:
# Insert new products
product1 = Product(name='Product 1', price=10.99)
product2 = Product(name='Product 2', price=20.99)
session.add_all([product1, product2])
session.commit()
print(f"Inserted Products: {product1.name}, {product2.name}")

Inserted Products: Product 1, Product 2


# Ajout d'un client

In [24]:
# Create a new customer
new_customer = Customer(name='azar')

# Add the new customer to the session
session.add(new_customer)

# Commit the transaction
session.commit()

print(f"Inserted Customer: {new_customer.name}")


Inserted Customer: azar


# Creation d'une nouvelle facture pour le client

In [25]:
# Create a new billing for the customer
new_billing = Billing(customer_id=new_customer.id)
session.add(new_billing)
session.commit()
print(f"Inserted Billing for Customer: {new_customer.name}")

Inserted Billing for Customer: azar


# Ajout de produit a la facture du client

In [26]:
# Add billing items for the customer
billing_item1 = BillingItem(billing_id=new_billing.id, product_id=product1.id, quantity=2)
billing_item2 = BillingItem(billing_id=new_billing.id, product_id=product2.id, quantity=1)
session.add_all([billing_item1, billing_item2])
session.commit()
print("Inserted Billing Items")

Inserted Billing Items


# Obtention des factures du client

In [27]:
def get_billing_items_by_customer_id(customer_id):
    customer = session.query(Customer).filter_by(id=customer_id).first()
    if customer:
        print(f"Billing items for customer: {customer.name}")
        for billing in customer.billings:
            print(f"Billing ID: {billing.id}, Date: {billing.date}")
            for item in billing.items:
                product = session.query(Product).filter_by(id=item.product_id).first()
                print(f"  Product: {product.name}, Quantity: {item.quantity}")
    else:
        print("Customer not found")

# Example usage:
customer_id = 1  # Replace with the actual customer ID
get_billing_items_by_customer_id(customer_id)


Billing items for customer: azar
Billing ID: 1, Date: 2024-07-09 19:50:34.252755
  Product: Product 1, Quantity: 2
  Product: Product 2, Quantity: 1
